<a href="https://colab.research.google.com/github/laddhashreya2000/Review_Analysis/blob/master/Customer_review_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import sys
sys.path.append('/content/drive/My Drive/NLP_research/Notebooks')

In [ ]:
from preprocessing import Process
from featureextraction import Feature
from pruning import Pruning
from sentiment import Opinion

... OK (Got response in 0.11 seconds)
Writing emoji data to /root/.demoji/codes.json ...
... OK


/usr/local/lib/python3.6/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [ ]:
def review_analysis(df, asm_support, compact, p_support):

  #preprocessing
  review = Process(df)
  review.lower() #convert all text to lower case
  review.demojify() #remove all emoji from data
  review.remove_1word() # 1 word reviews do not contain sentiment towards any feature, so all one word reviews are removed
  review.remove_2word() #those 2 word reviews that do not contain any noun, i.e any feature of product are removed
  
  #Feature extraction
  review = Feature(review.df)
  review.extract_nouns() # extract all nouns from the text
  review.ass_rules(support=asm_support) #Apply apriori algorithm to extract frequently mentioned features from nouns
  review.get_features() # extracts feature list from the dataset after applying asm with appropriate support

  #Applying feature pruning to extract meaningful features from the list extracted above
  review = Pruning(review.df , review.features)
  review.compactness_pruning(per = compact) #appplying compactness pruning on the two word features, which extracts a list of the values of features, their total appearance and the number of sentences where they are compact
  review.redundancy_pruning(thresh=p_support) #applies pruning for 1 word feature, to determine which are meaningful or not. Check cluster analysis of customer reviews article by Prof. Gourab Nath.
  review.final_list()#get final list of meaningful features for our product
  review.correct() #correct the nouns which are incorrect forms of the features in the dataset

  #Opinion extraction
  review = Opinion(review.df, review.final_dict, review.one_word_feature, review.two_word_features)
  review.sentiment_score_one_word()#extract sentiment scores of one word features
  review.sentiment_score_two_word()#extract sentiment scores of two word features

  return

In [ ]:
import pandas as pd
df=pd.read_csv('/content/drive/My Drive/NLP_research/Dataset/reviews.csv')
print(df.shape)
df.head(1)

(1425, 7)


,id,profileName,text,date,title,rating,images
0,R1XPFOGBHSG8K4,Akash peelwa,\n PLZZ read this complete information before...,Reviewed in India on 9 June 2020,PLZZ read this before Wasting your money.,1,NaN


In [ ]:
review_analysis(df['text'], asm_support=0.01, compact=0.44, p_support=40)